In [4]:

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer
from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms.minimum_eigen_solvers import VQE
from qiskit.circuit import Parameter
from qiskit.opflow import I, X, Z, Y, StateFn
from qiskit.utils import QuantumInstance

# Define la molécula y sus operadores
mol = 'H .0 .0 -1.0; H .0 .0 1.0'
num_qubits = 4
hamiltonian = (-1.04) * (I ^ I ^ I ^ I) + 0.39 * (Z ^ I ^ I ^ I) \
              - 0.39 * (I ^ Z ^ I ^ I) - 0.39 * (I ^ I ^ Z ^ I) \
              + 0.18 * (Z ^ Z ^ I ^ I) + 0.18 * (Z ^ I ^ Z ^ I) \
              + 0.18 * (Z ^ I ^ I ^ Z) + 0.18 * (I ^ Z ^ Z ^ I) \
              + 0.18 * (I ^ Z ^ I ^ Z) + 0.18 * (I ^ I ^ Z ^ Z) \
              - 0.05 * (X ^ X ^ X ^ X) - 0.05 * (Y ^ Y ^ X ^ X) \
              - 0.05 * (X ^ X ^ Y ^ Y) + 0.05 * (Y ^ Y ^ Y ^ Y) \
              + 0.28 * (Z ^ Z ^ Z ^ I) + 0.28 * (Z ^ I ^ Z ^ Z) \
              - 0.13 * (X ^ Y ^ Y ^ X) - 0.13 * (Y ^ X ^ X ^ Y)
              
# Define el ansatz 
def ansatz(params, qubits):
    circ = QuantumCircuit(qubits)
    for i in range(num_qubits):
        circ.rx(params[3*i], i)
        circ.ry(params[3*i+1], i)
        circ.rz(params[3*i+2], i)
    return circ

# Define la energía
def energy(params):
    circ = ansatz(params, qubits)
    psi = StateFn(circ, is_measurement=True)
    H_psi = psi.adjoint().compose(hamiltonian.compose(psi))
    # Obtén el valor esperado del Hamiltoniano
    result = H_psi.eval().real
    return result

# Inicializa los parámetros, el optimizador y los dispositivos
params = [Parameter(f"theta_{i}") for i in range(3*num_qubits)]
optimizer = SPSA(maxiter=100)
qubits = QuantumRegister(num_qubits)
shots = 1024
dev = QuantumInstance(Aer.get_backend('aer_simulator'), shots=shots)

# Ejecuta el VQE
vqe = VQE(ansatz=ansatz, optimizer=optimizer, parameter_values=params, 
    quantum_instance=dev, include_custom=True)
energy, params_optimized = vqe.compute_minimum_eigenvalue(hamiltonian)
print(f" E = {energy:.5f}")

TypeError: VQE.__init__() got an unexpected keyword argument 'parameter_values'

En este ejemplo se define una molécula de hidrógeno y se crea un Hamiltoniano de dos qubits, seguido de la definición de un ansatz de circuito y una función de energía que calcula el valor esperado del Hamiltoniano. A continuación, se inicializan los parámetros, el optimizador y se define el dispositivo, y finalmente se ejecuta el VQE y se imprime el valor de la energía obtenido.

Recuerda que este es solo un ejemplo básico y que la implementación del algoritmo del VQE puede ser muy personalizada en función del problema específico que se esté abordando.

---

Aquí te dejo un ejemplo básico de cómo utilizar el VQE en Qiskit para optimizar un modelo de regresión lineal simple:

In [ ]:

import numpy as np
from qiskit import Aer
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import Gradient
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import COBYLA
from qiskit_machine_learning.algorithms import VQC
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Carga los datos de Boston Housing
boston = load_boston()
X, y = boston.data, boston.target

# Escala las características de 0 a 1
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Separa los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Definir el circuito de ansatz y el modelo de clasificación cuántica
feature_map = RealAmplitudes(num_qubits=4, entanglement='full', reps=1)
qc = feature_map.compose(RealAmplitudes(num_qubits=4, entanglement='full', reps=3))
optimizer = COBYLA(maxiter=50)
q_instance = QuantumInstance(backend=Aer.get_backend('qasm_simulator'),
                             shots=1000, seed_simulator=42, seed_transpiler=42)

# Define la función de costo y el gradiente
def cost_function(theta):
    qc = feature_map.bind_parameters(theta)
    vqc = VQC(optimizer=optimizer, feature_map=qc, ansatz=qc, gradient=Gradient())
    vqc.fit(X_train, y_train)
    return vqc.predict(X_test)

# Optimiza los parámetros del modelo usando VQE
theta_initial = np.random.uniform(0, 2 * np.pi, qc.num_parameters)
vqe = VQE(ansatz=qc, optimizer=optimizer, quantum_instance=q_instance)
theta_optimized = vqe.compute_minimum_eigenvalue(cost_function, initial_point=theta_initial)[0]

# Prueba el modelo con los datos de prueba
qc = feature_map.bind_parameters(theta_optimized)
vqc = VQC(optimizer=optimizer, feature_map=qc, ansatz=qc, gradient=Gradient())
vqc.fit(X_train, y_train)
score = vqc.score(X_test, y_test)
print('Score: {:.2f}'.format(score))


En este ejemplo, se utiliza el VQE para optimizar un modelo de regresión lineal simple utilizando un circuito de ansatz variacional (representado por RealAmplitudes), y se utiliza el algoritmo VQC (Variational Quantum Classifier) para entrenar y predecir los resultados. La función de costo utilizada en este caso es simplemente la función de predicción del modelo, que es cuántica ya que se usa la quantum feature map y el ansatz.

Recuerda que este es solo un ejemplo muy básico y que la implementación del VQE en el contexto del machine learning puede variar dependiendo de la aplicación específica, y puede requerir un enfoque más sofisticado para asegurar la extrapolación adecuada de los resultados.

---
Tienes razón, me disculpo por el error en mi respuesta anterior. En la implementación de machine learning que te mostré anteriormente, la elección del algoritmo del VQE puede no ser explícita. En su lugar, se utiliza el algoritmo VQC, que es una variante del VQE que se utiliza específicamente en aplicaciones de clasificación.


In [3]:

from qiskit import Aer
from qiskit.algorithms.optimizers import L_BFGS_B
from qiskit.opflow import PauliSumOp
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE
from qiskit.utils import QuantumInstance
import numpy as np

# Define el problema de optimización
matrix = np.array([[0.5, 0.2], [0.2, -0.5]])
pauli_op = PauliSumOp.from_list([(matrix, "II"), (matrix, "IZ"), (matrix, "ZI"), (matrix, "ZZ")])

# Define el ansatz de circuito cuántico
depth = 2
entanglement = 'circular'
ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz', reps=depth, entanglement=entanglement)

# Define el algoritmo VQE
optimizer = L_BFGS_B()
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

algorithm = VQE(ansatz=ansatz,
                optimizer=optimizer,
                quantum_instance=quantum_instance)

# Encuentra el mínimo global de la función de costo
result = algorithm.compute_minimum_eigenvalue(operator=pauli_op)
print(result)

ValueError: setting an array element with a sequence

En este ejemplo, la función de costo está definida por una matriz de 2 x 2, y se utiliza el algoritmo del VQE para encontrar su mínimo global. El ansatz que se implementa es "TwoLocal", que es muy común en las soluciones de VQE y utiliza dos capas de puertas rg para rotación y puertas cz para entrelazamiento. El algoritmo de optimización utilizado aquí es L_BFGS_B, pero se pueden utilizar otros algoritmos, dependiendo del tipo de problema que se esté abordando.

Espero que esto ayude a aclarar cualquier confusión y lamento cualquier inconveniente causado por mi respuesta anterior.

¡Hola de nuevo! Te comparto el código de este mismo ejemplo en Qiskit:

```python
from qiskit import QuantumCircuit, Aer
import numpy as np

variables = 3
shots = 1000
backend = Aer.get_backend('qasm_simulator')
circ = QuantumCircuit(variables, variables)

def phi(w):
circ.rx(w[0], 0)
circ.rx(w[1], 1)
circ.rx(w[2], 2)

def circ1(w):
phi(w)
circ.measure([0, 1, 2], [0, 1, 2])
result = backend.run(circ, shots=shots).result().get_counts(circ)
expval = 1
for key in result:
if key != '000':
expval -= result[key]/shots
return expval

def circ2(w):
phi(w)
circ.measure([0, 1, 2], [0, 1, 2])
result = backend.run(circ, shots=shots).result().get_counts(circ)
expval = 1
for key in result:
if key[1] == '1':
expval -= result[key]/shots
return expval

def circ3(w):
phi(w)
circ.measure([0, 1, 2], [0, 1, 2])
result = backend.run(circ, shots=shots).result().get_counts(circ)
expval = 1
for key in result:
if key[2] == '1':
expval -= result[key]/shots
return expval

def circ4(w):
phi(w)
circ.measure([0, 1, 2], [0, 1, 2])
result = backend.run(circ, shots=shots).result().get_counts(circ)
expval = 1
for key in result:
if key[0] == '1' and key[1] == '1':
expval -= result[key]/shots
return expval

def circ5(w):
phi(w)
circ.measure([0,

In [4]:
# Importamos las librerías necesarias
from qiskit import Aer, QuantumCircuit, assemble
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA

# Creamos el circuito cuántico a optimizar
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Definimos la instancia para el algoritmo VQE
vqe = VQE(ansatz=qc, optimizer=COBYLA(), quantum_instance=Aer.get_backend('qasm_simulator'))

# Ejecutamos el algoritmo y obtenemos el resultado óptimo
result = vqe.
print('El resultado óptimo es:', result.eigenvalue.real)

AttributeError: 'VQE' object has no attribute 'run'

In [ ]:

import numpy as np
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram

# Para recrear el circuito en Qiskit, podemos utilizar la función `QuantumCircuit` para definir nuestro circuito cuántico, y las puertas cuánticas disponibles en Qiskit como `rx`, `pz` e `id` para construir las operaciones.

# Aquí te muestro el código traducido a Qiskit:

variables = 3
shots = 1000
backend = Aer.get_backend('qasm_simulator')

qc = QuantumCircuit(variables)

def phi(qc, w):
    qc.rx(w[0], 0)
    qc.rx(w[1], 1)
    qc.rx(w[2], 2)

def circ1(qc, w):
    phi(qc, w)
    qc.id([0,1,2])
    return qc

def circ2(qc, w):
    phi(qc, w)
    qc.p(-1, 1)
    qc.id([0,2])
    return qc

def circ3(qc, w):
    phi(qc, w)
    qc.id([0,1])
    qc.p(-1, 2)
    return qc

def circ4(qc, w):
    phi(qc, w)
    qc.p(-1, 0)
    qc.p(-1, 1)
    qc.id([2])
    return qc

def circ5(qc, w):
    phi(qc, w)
    qc.p(-1, 1)
    qc.p(-1, 2)
    qc.id([0])
    return qc

def value(w):
    counts_circ1 = execute(circ1(QuantumCircuit(variables), w), backend, shots=shots).result().get_counts()
    counts_circ2 = execute(circ2(QuantumCircuit(variables), w), backend, shots=shots).result().get_counts()
    counts_circ3 = execute(circ3(QuantumCircuit(variables), w), backend, shots=shots).result().get_counts()
    counts_circ4 = execute(circ4(QuantumCircuit(variables), w), backend, shots=shots).result().get_counts()
    counts_circ5 = execute(circ5(QuantumCircuit(variables), w), backend, shots=shots).result().get_counts()

    expval_circ1 = (counts_circ1.get('000', 0) + counts_circ1.get('001', 0) + counts_circ1.get('010', 0) + counts_circ1.get('011', 0) + counts_circ1.get('100', 0) + counts_circ1.get('101', 0) + counts_circ1.get('110', 0) + counts_circ1.get('111', 0) - shots) / shots
    expval_circ2 = (counts_circ2.get('000', 0) + counts_circ2.get('001', 0) + counts_circ2.get('010', 0) + counts_circ2.get('011', 0) - counts_circ2.get('100', 0) - counts_circ2.get('101', 0) - counts_circ2.get('110', 0) - counts_circ2.get('111', 0)) / shots
    expval_circ3 = (counts_circ3.get('000', 0) + counts_circ3.get('001', 0) - counts_circ3.get('010', 0) - counts_circ3.get('011', 0) + counts_circ3.get('100', 0) + counts_circ3.get('101', 0) - counts_circ3.get('110',

In [2]:
from qiskit import QuantumCircuit, execute, Aer, Gr
import numpy as np

# Crea un circuito cuántico
circ = QuantumCircuit(3, 3)

# Definir función phi
def phi(w):
    circ.rx(w[0], 0)
    circ.rx(w[1], 1)
    circ.rx(w[2], 2)

# Define las funciones circulares
def circ1(w):
    phi(w)
    circ.measure(range(3),range(3))
    result = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000).result().get_counts(circ)
    return result['000']/1000

def circ2(w):
    phi(w)
    circ.measure([0,2],[0,2])
    circ.measure(1,1)
    result = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000).result().get_counts(circ)
    return (result['000'] + result['010'] - result['001'] - result['011'])/1000

def circ3(w):
    phi(w)
    circ.measure([0,1],[0,1])
    circ.measure(2,2)
    result = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000).result().get_counts(circ)
    return (result['000'] + result['100'] - result['001'] - result['101'])/1000

def circ4(w):
    phi(w)
    circ.cx(0,1)
    circ.measure(range(2),range(2))
    result = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000).result().get_counts(circ)
    return (result['00'] + result['11'] - result['01'] - result['10'])/1000

def circ5(w):
    phi(w)
    circ.cx(0,1)
    circ.cx(0,2)
    circ.cx(1,2)
    circ.measure(range(3),range(3))
    result = execute(circ, Aer.get_backend('qasm_simulator'), shots=1000).result().get_counts(circ)
    return (result['000'] + result['111'] - result['001'] - result['010'] - result['100'] - result['011'] - result['101'] - result['110'])/1000

# Define la función de pérdida
def value(w):
    return 3*circ1(w)/4 + circ2(w)/4 - 3*circ3(w)/4 - circ4(w) + 3*circ5(w)/4

# Define la función de gradiente
gradient_fn_w = GradientDescent.gradient(value)

AttributeError: 'AerProvider' object has no attribute 'Gradient'